## The `LinearAgent` class

The `LinearAgent` class represents dynamical systems driven by time-varying state-update and output equations of the form
\begin{align}
x(k+1) & = A(k) x(k) + B(k) u(k) + f(k),\\
y(k) & = C(k) x(k) + D(k) u(k) + g(k).
\end{align}

Here, $x \in \mathbb{R}^{n_x}$ is the state vector, $u \in \mathbb{R}^{n_u}$ is the vector of control inputs, and $y \in \mathbb{R}^{n_y}$ represents the vector of system's outputs. These variables are subject to time-varying constraints of the form
\begin{align}
& x_{\text{min}}(k) \le x(k) \le x_{\text{max}}(k),\\
& u_{\text{min}}(k) \le u(k) \le u_{\text{max}}(k),\\
& y_{\text{min}}(k) \le y(k) \le y_{\text{max}}(k).
\end{align}


Objects of the class serve as placeholders for storing the dynamics (represented by matrices $A$, $B$, $f$, $C$, $D$, and $g$), constraints, as well as information about the control objectives. In particular, the optimal control inputs are computed by solving the following optimal control problem:
\begin{align}
\min \ &
(x(N) - x_{\text{ref}}(N))^T Q_{\text{x}}(N) (x(N) - x_{\text{ref}}(N)) + \cdots\\
& \qquad \cdots + \sum_{k=0}^{N-1} (x(k) - x_{\text{ref}}(k))^T Q_{\text{x}}(k) (x(k) - x_{\text{ref}}(k)) + \cdots\\
& \qquad \cdots + \sum_{k=0}^{N-1} (u(k) - u_{\text{ref}}(k))^T Q_{\text{u}}(k) (u(k) - u_{\text{ref}}(k)) +\cdots\\
& \qquad \cdots + \sum_{k=0}^{N-1} (y(k) - y_{\text{ref}}(k))^T Q_{\text{y}}(k) (y(k) - y_{\text{ref}}(k))\\
\text{s.t.} \
& x(k+1) = A(k) x(k) + B(k) u(k) + f(k), \ k = 0, \ldots, N-1,\\
& y(k) = C(k) x(k) + D(k) u(k) + g(k), \ k = 0, \ldots, N-1,\\
& x_{\text{min}}(k) \le x(k) \le x_{\text{max}}(k), \ k = 0, \ldots, N,\\
& u_{\text{min}}(k) \le u(k) \le u_{\text{max}}(k), \ k = 0, \ldots, N-1,\\
& y_{\text{min}}(k) \le y(k) \le y_{\text{max}}(k), \ k = 0, \ldots, N-1.
\end{align}

The objective function employs time-varying penalty matrices $Q_{\text{x}} \in \mathbb{R}^{n_{x} \times n_x}$, $Q_{\text{u}} \in \mathbb{R}^{n_{u} \times n_u}$, and $Q_{\text{y}} \in \mathbb{R}^{n_{y} \times n_y}$ which penalize the deviation of respective signals from their references.

Instances of the `LinearAgent` class are generated by calling the constructor with following mandatory inputs:
* `nx` - number of states
* `nu` - number of control inputs
* `ny` - number of outputs
* `N` - prediction horizon

Let's create a sample agent. Its dynamics will represented by 4 states:
* `x1`: speed in the x-axis
* `x2`: position in the x-axis
* `x3`: speed in the y-axis
* `x4`: position in the y-axis

two control inputs:
* `u1`: accelleration in the x-axis
* `u2`: accelleration in the y-axis

and two outputs:
* `y1`: position in the x-axis
* `y2`: position in the y-axis

The dynamics in both axis is decoupled and each is represented by a discrete-time double integrator. The agent's movements will be optimized by a model predictive scheme with prediction horizon `N`.

In [15]:
nx = 4; nu = 2; ny = 2; N = 10;
agent = optiplan.LinearAgent('nx', nx, 'nu', nu, 'ny', ny, 'PredictionHorizon', N)

agent = 

  LinearAgent with properties:

       A: [4x4x10 optiplan.AgentSignal]
       B: [4x2x10 optiplan.AgentSignal]
       f: [4x1x10 optiplan.AgentSignal]
       C: [2x4x10 optiplan.AgentSignal]
       D: [2x2x10 optiplan.AgentSignal]
       g: [2x1x10 optiplan.AgentSignal]
      nx: 4
      nu: 2
      ny: 2
       N: 10
       X: [4x1x11 optiplan.AgentSignal]
       U: [2x1x10 optiplan.AgentSignal]
       Y: [2x1x10 optiplan.AgentSignal]
    Size: [2x1x10 optiplan.AgentSignal]

The agent has following properties:
* `A`, `B`, `f`: matrices of the state-update equation $x(k+1) = A(k)x(k) + B(k)u(k) + f(k)$
* `C`, `D`, `g`: matrices of the output equation $y(k) = C(k)x(k) + D(k)u(k) + g(k)$
* `Size`: size of the agent given as a vector of `width` (in the x-axis) and `height` (in the y-axis)
* `X`: open-loop predictions of system's states, i.e., $X = [x_0, x_1, \ldots, x_N]$
* `U`: open-loop predictions of system's inputs, i.e., $U = [u_0, u_1, \ldots, u_{N-1}]$
* `Y`: open-loop predictions of system's outputs, i.e., $Y = [y_0, y_1, \ldots, y_{N-1}]$

By default, all these properties are *parameters* which means that they can be time-varying. This is indicated by the value being set to `'parameter'`:

In [16]:
agent.A.Value

ans =

parameter

To tell the tool that the respective quantity is fixed throughout the prediction horizon, set its `Value` property to the desired value. Here, we will indicate that the dynamics is indeed constant. To save typing, we will load some demo data:

In [17]:
Ts = 0.25;  % sampling time
demo = optiplan.LinearAgent.demo2Ddata(Ts);

In [18]:
agent.A.Value = demo.A;
agent.B.Value = demo.B;
agent.f.Value = demo.f;
agent.C.Value = demo.C;
agent.D.Value = demo.D;
agent.g.Value = demo.g;

The `agent.X`, `agent.Y`, and `agent.U` signals are special since they allow you to specify additional properties, such as min/max bounds, reference values to be tracked, and matrices which penalize the deviation of the respective signals from their reference values:

In [19]:
agent.X

ans = 

  4x1x11 AgentSignal array with properties:

          Max: 'parameter'
      Penalty: 'parameter'
    Reference: 'parameter'
          Min: 'parameter'

This output means that the agent's state vector will be subjecto to bounds $x_{\text{min}} \le x_k \le x_{\text{max}}$ for $k = 0, \ldots, N$, and both the lower as well as the upper bound are parametric (i.e., they are time-varying, not known at the present time, but their values will be provided during the implementation).

Additionally, the difference between the state vector and the value in the `agent.X.Reference` will be minimized at each prediction step by adding $(x_k - x_{\text{ref}})^T Q (x_k - x_{\text{ref}})$ to the objective function. Here, $x_k$ is the prediction of the state vector at the $k$-th step, $x_{\text{ref}}$ is what's stored in `agent.X.Reference`, and `agent.X.Penalty` specifies the value of the penalty matrix $Q$.

If you wish, you can declare some of these properties to be constant. Let's use constant min/max bounds and the penalty matrix, but keep the reference parametric:

In [20]:
agent.X.Min = demo.xmin;
agent.X.Max = demo.xmax;
agent.X.Penalty = zeros(agent.nx);
agent.X

ans = 

  4x1x11 AgentSignal array with properties:

          Max: [4x1 double]
      Penalty: [4x4 double]
    Reference: 'parameter'
          Min: [4x1 double]

Sidenote: the tool automatically checks for correct dimensions of the signals:

In [21]:
agent.X.Min = [1; 2; 3]

Value must be a 4x1 vector/matrix.


To use the agent for collision avoidance, you should set its size by setting `agent.Size.Value` to a $n_y \times 1$ vector (if the size is fixed), or to `parameter` (if the size can be changed during the simulation). Note that the agent's outputs are implicitly assumed to be the agent's positions.

In [22]:
x_width = 1;
y_height = 1;
agent.Size.Value = [x_width; y_height];

It's easy to loose track about which properties are fixed and which are parametric. No need to wory, though. Call the agent's `listParameters()` method to get the info:

In [23]:
agent.listParameters()

agent.U.Max        [2 1 10]
agent.U.Min        [2 1 10]
agent.U.Penalty    [2 2 10]
agent.U.Reference  [2 1 10]
agent.X.Reference  [4 1 11]
agent.Y.Max        [2 1 10]
agent.Y.Min        [2 1 10]
agent.Y.Penalty    [2 2 10]
agent.Y.Reference  [2 1 10]

You can switch a property from parametric to fixed at any time. To declare it as parametric, set its value to `'parameter'`. Fixed properties are indicated by double values. Lets fix the penalty matrices for now:

In [24]:
agent.U.Penalty = demo.Qu;
agent.Y.Penalty = demo.Qy;